In [ ]:
import numpy as np
from tqdm import tqdm
import scipy.sparse as sparse
import scipy.sparse.linalg as arp
import warnings
import itertools

## Observables

In [20]:
Pauli_I = np.array([[1,0],[0,1]],dtype=np.complex64)
Pauli_x = np.array([[0,1],[1,0]],dtype=np.complex64)
Pauli_y = np.array([[0,-1j],[1j,0]],dtype=np.complex64)
Pauli_z = np.array([[1,0],[0,-1]],dtype=np.complex64)

In [21]:
def generate_basis_observables(num_bit):
    permutation_list = list(itertools.product(list(range(1,4)),repeat=num_bit))
    # float_observables = []
    # observables = []
    print(len(permutation_list))
    for k in tqdm(range(0,len(permutation_list))):
        index_list = []
        if permutation_list[k][0] == 0:
            tmp = Pauli_I
        elif permutation_list[k][0] == 1:
            tmp = Pauli_x
            index_list += [1, 0, 0]
        elif permutation_list[k][0] == 2:
            tmp = Pauli_y
            index_list += [0, 1, 0]
        elif permutation_list[k][0] == 3:
            tmp = Pauli_z
            index_list += [0, 0, 1]
        for i in range(1,num_bit):
            if permutation_list[k][i] == 0:
                tmp = np.kron(tmp,Pauli_I)
            elif permutation_list[k][i] == 1:
                tmp = np.kron(tmp,Pauli_x)
                index_list += [1, 0, 0]
            elif permutation_list[k][i] == 2:
                tmp = np.kron(tmp,Pauli_y)
                index_list += [0, 1, 0]
            elif permutation_list[k][i] == 3:
                tmp = np.kron(tmp,Pauli_z)
                index_list += [0, 0, 1]
        observable = tmp
        np.save(str(num_bit) + 'qubit/observable' + str(num_bit) + str(k), observable)
        # observables.append(observable)

        observable = observable.reshape(observable.shape[0]*observable.shape[1])
        observable_real = observable.real
        observable_imag = observable.imag
        observable = np.concatenate((observable_real,observable_imag))
        np.save(str(num_bit) + 'qubit/float_observable' + str(num_bit) + str(k), observable)
        # float_observables.append(observable)
        # np.save(str(num_bit) + 'qubit/observable_index'+ str(num_bit) + str(k), np.array(index_list))

    return 0

In [22]:
num_bit = 6
_ = generate_basis_observables(num_bit)

729


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 729/729 [00:01<00:00, 461.21it/s]


## State Generation

In [24]:
sxx = Pauli_x
syy = Pauli_y
szz = Pauli_z

In [25]:
def exact_E_rand_Js(L, Js, h):
    """For comparison: obtain ground state energy from exact diagonalization.
    Exponentially expensive in L, only works for small enough `L` <~ 20.
    """
    if L >= 20:
        warnings.warn("Large L: Exact diagonalization might take a long time!")
    # get single site operaors
    sx = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]))
    sz = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]))
    id = sparse.csr_matrix(np.eye(2))
    sx_list = []  # sx_list[i] = kron([id, id, ..., id, sx, id, .... id])
    sz_list = []
    for i_site in range(L):
        x_ops = [id] * L
        z_ops = [id] * L
        x_ops[i_site] = sx
        z_ops[i_site] = sz
        X = x_ops[0]
        Z = z_ops[0]
        for j in range(1, L):
            X = sparse.kron(X, x_ops[j], 'csr')
            Z = sparse.kron(Z, z_ops[j], 'csr')
        sx_list.append(X)
        sz_list.append(Z)
    H_x = sparse.csr_matrix((2**L, 2**L))
    H_zz = sparse.csr_matrix((2**L, 2**L))
    for i in range(L - 1):
        rand_J = Js[i]
        H_zz = H_zz + rand_J*sz_list[i] * sz_list[(i + 1) % L]
    for i in range(L):
        H_x = H_x + sx_list[i]
    H = -H_zz - h * H_x
    E, V = arp.eigsh(H, k=2, which='SA', return_eigenvectors=True, ncv=20)
    return V[:,0]

## Probability Generation

In [27]:
vs = []
for j in range(0,3**num_bit):
    observable = np.load(str(num_bit)+'qubit/observable'+str(num_bit)+str(j) + '.npy')
    vs.append(np.linalg.eig(observable)[1])

In [28]:
def exact_E_rand_Js_probs(state,L,vs):
    values = []
    for j in range(0,3**num_bit):
        tmp = []
        for k in range(0,2**num_bit):
            tmp.append(np.abs(np.inner(state.conj().T,vs[j][:,k]))**2)
        values.append(tmp)
    return values

## Dataset Generation

In [29]:
L = num_bit
h = 1
num_states= 2000
for i in tqdm(range(0,num_states)):
    Js = np.random.random(L)*2-1
    state = exact_E_rand_Js(L, Js, h)
    probs = exact_E_rand_Js_probs(state,L,vs)
    np.save(str(L)+'qubit/Ising_ground_state_'+str(L)+'qubit_probs_random_'+ str(i)+'.npy',probs)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [09:37<00:00,  3.46it/s]
